In [7]:
from bs4 import BeautifulSoup
import urllib2
import string
import sqlite3
from functools import reduce
import numpy as np
import pandas as pd

In [71]:
CURRENT_SEASON = "2018"
NUM_YEARS = 20
MAX_CAREER_LENGTH = 26

In [116]:
def pro_football_focus_helper(url, columns):
    index = 0
    ls = []
    while(True):
        page = urllib2.urlopen(url + "&offset=" + str(index))
        soup = BeautifulSoup(page, 'lxml')
        table_div = soup.find("div", {"id" : "div_results"})
        
        try:
            body = table_div.find("tbody")
        except:
            return ls
        
        rows = body.findAll("tr", {"class": None})

        [ls.append(i) for i in map(lambda x: [x.find("td", {"data-stat": column}).text for column in columns], rows)]
        index += 100

def get_all_players():
    url = "https://www.pro-football-reference.com/play-index/psl_finder.cgi?request=1&match=total&draft=1"
    url += "&year_min=" + str(int(CURRENT_SEASON) - NUM_YEARS + 1) + "&year_max=" + CURRENT_SEASON + "&season_start=1&season_end=-1&pos%5B%5D=wr&draft_year_min=1936"
    url += "&draft_year_max=2019&draft_slot_min=1&draft_slot_max=500&draft_pick_in_round=pick_overall&conference=any"
    url += "&draft_pos%5B%5D=wr&c1stat=rec_yds&c1comp=gt&c1val=900&c5val=1.0&order_by=rec_yds"
    
    return map(lambda x: x[0], pro_football_focus_helper(url, ["player"]))
        
def get_all_seasons(): 
    url = "https://www.pro-football-reference.com/play-index/psl_finder.cgi?request=1&match=single"
    url += "&year_min=" + str(int(CURRENT_SEASON) - NUM_YEARS + 1) + "&year_max=" + CURRENT_SEASON + "&season_start=1&season_end=-1&pos%5B%5D=wr&draft_year_min=1936"
    url += "&draft_year_max=2019&draft_slot_min=1&draft_slot_max=500&draft_pick_in_round=pick_overall&conference=any"
    url += "&draft_pos%5B%5D=wr&c1stat=rec_yds&c1comp=gt&c1val=900&c5val=1.0&order_by=rec_yds"
    
    url2 = "https://www.pro-football-reference.com/play-index/psl_finder.cgi?request=1&match=combined"
    url2 += "&year_min=" + str(int(CURRENT_SEASON) - NUM_YEARS - MAX_CAREER_LENGTH + 1) + "&year_max=" + CURRENT_SEASON + "&season_start=1&season_end=-1&pos%5B%5D=wr&draft_year_min=1936"
    url2 += "&draft_year_max=2019&draft_slot_min=1&draft_slot_max=500&draft_pick_in_round=pick_overall&conference=any"
    url2 += "&draft_pos%5B%5D=wr&c1stat=rec_yds&c1comp=gt&c1val=900&c5val=1.0&order_by=rec_yds"
    
    
    data = pro_football_focus_helper(url, ["player", "year_id", "psl_draft_info", "rec_yds"])
    drafted_year = pro_football_focus_helper(url2, ["player", "year_min"])
    drafted_year_dict = {d[0]: d[1] for d in drafted_year}
    data = [[d[0], int(d[1]) - int(drafted_year_dict[d[0]]) + 1, 0, 1, d[3]] if d[2] == "" else [d[0], int(d[1]) - int(drafted_year_dict[d[0]]) + 1, d[2].split("-")[0], d[2].split("-")[1], d[3]] for d in data]
    return pd.DataFrame(data, columns =['Name', 'Year', 'Round', 'Pick', 'Receiving Yards'], dtype = float)

In [167]:
all_seasons = get_all_seasons()
all_players = get_all_players()
all_seasons = all_seasons.sort_values(by=["Name", "Year"])

prev_name = ""
first_seasons = pd.DataFrame([], columns =['Name', 'Year', 'Round', 'Pick', 'Receiving Yards'], dtype = float)
for idx, season in all_seasons.iterrows():
    if(season['Name']) == "Steve Smith":
        print season['Year']
    if season['Name'] != prev_name:
        first_seasons = first_seasons.append(season)
        prev_name = season['Name']    
        
# TODO differentiate between players with the same name (AKA Steve Smith)

-3.0
-1.0
0.0
1.0
2.0
3.0
3.0
5.0
6.0
8.0


In [166]:
first_seasons.sort_values(by=["Year"])

,Name,Year,Round,Pick,Receiving Yards
263,Steve Smith,-3.0,3.0,74.0,1110.0
320,A.J. Green,1.0,1.0,4.0,1057.0
347,Marques Colston,1.0,7.0,252.0,1038.0
190,Michael Clayton,1.0,1.0,15.0,1193.0
418,Eddie Royal,1.0,2.0,42.0,980.0
241,Michael Thomas,1.0,2.0,47.0,1137.0
329,Mike Evans,1.0,1.0,7.0,1051.0
120,Odell Beckham,1.0,1.0,12.0,1305.0
497,DeSean Jackson,1.0,2.0,49.0,912.0
376,Kelvin Benjamin,1.0,1.0,28.0,1008.0
